### Uses Paper Trade - test money
##### https://alpaca.markets/docs/api-documentation/api-v2/

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import time, datetime
import pandas as pd

import alpha, database
from populate import *
from portfolios import *
from state import *

DataStore = database.DataStore()
SP500 = SP500()

In [2]:
st = State(DataStore, 'AAPL', indicators=['ADX', 'CCI', 'EMA'])
st.df.head(20)

,adjusted close,ADX,CCI,EMA
2021-06-14,130.48,17.5953,162.2648,126.5240
2021-06-11,127.35,15.7934,81.2796,126.2360
2021-06-10,126.11,17.0469,81.5230,126.1707
2021-06-09,127.13,18.4396,106.7193,125.9820
2021-06-08,126.74,19.1520,106.1008,125.7091
2021-06-07,125.9,19.9435,4.7470,125.5111
2021-06-04,125.89,20.8202,-25.1129,125.3647
2021-06-03,123.54,21.5606,-128.5311,125.6524
2021-06-02,125.06,20.3478,-71.7412,125.8684
2021-06-01,124.28,19.8471,-104.0564,126.2214


In [6]:
# from ray import tune
# from ray.rllib.agents.ppo import PPOTrainer
# tune.run(PPOTrainer, config={"env": "CartPole-v0", "log_level": "INFO", "framework": "torch"})

(pid=None) 'C:\Users\albeadam\OneDrive' is not recognized as an internal or external command,
(pid=None) operable program or batch file.
(pid=None) 'C:\Users\albeadam\OneDrive' is not recognized as an internal or external command,
(pid=None) operable program or batch file.
(pid=None) 'C:\Users\albeadam\OneDrive' is not recognized as an internal or external command,
(pid=None) operable program or batch file.
(pid=None) 'C:\Users\albeadam\OneDrive' is not recognized as an internal or external command,
(pid=None) operable program or batch file.
(pid=None) 'C:\Users\albeadam\OneDrive' is not recognized as an internal or external command,
(pid=None) operable program or batch file.
(pid=None) 'C:\Users\albeadam\OneDrive' is not recognized as an internal or external command,
(pid=None) operable program or batch file.
(pid=None) 'C:\Users\albeadam\OneDrive' is not recognized as an internal or external command,
(pid=None) operable program or batch file.
(pid=None) 'C:\Users\albeadam\OneDrive' i